In [ ]:
# 참고 : https://github.com/seongheum-ssu/2021-1-2150054901/blob/main/exp/2021-05-13
# -*- coding: utf-8 -*-
import cv2
import random
from google.colab.patches import cv2_imshow # Google Colab.
from tqdm import tqdm

# Parameters
TH1 = 0.5  # Confidence
TH2 = 0.25 # Non-maximum Suppression

# CNN model
config = '/content/drive/MyDrive/Colab Notebooks/ComputerVision/project/yolov3.cfg' 
model = '/content/drive/MyDrive/Colab Notebooks/ComputerVision/project/yolov3.weights' 
classLabels = '/content/drive/MyDrive/Colab Notebooks/ComputerVision/project/coco.names'

In [ ]:
# from darknet.py
def fun_get_colors(names):
    """
    Create a dict with one random BGR color for each
    class name
    """
    random.seed(1)
    return {name: (
        random.randint(0, 255),
        random.randint(0, 255),
        random.randint(0, 255)) for name in names}

In [ ]:
# main(동영상 저장과 경고 알림 추가)
if __name__ == "__main__":
    # Load CNN model trained over MS COCO DB
    net = cv2.dnn.readNetFromDarknet(config, model)
    layerNames = net.getLayerNames()
    layerOutputs = [layerNames[i[0] - 1] for i in net.getUnconnectedOutLayers()]
    with open(classLabels, 'rt') as file:
        classNames = file.read().strip('\n').split('\n')
    classColors = fun_get_colors(classNames)
    
    cap = cv2.VideoCapture('/content/drive/MyDrive/Colab Notebooks/ComputerVision/project/videos/use_cellphone3.mp4')
    total_frame = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    for now_frame in tqdm(range(total_frame)):
        ret, img_ori = cap.read()

        img_ori = cv2.resize(img_ori, dsize=(0, 0), fx=0.5, fy=0.5)

        img = img_ori.copy()
        # Run CNN: input image to output tensor
        try:
            blob = cv2.dnn.blobFromImage(img, 1/255, (416, 416), swapRB=1)
            # blob = cv2.dnn.blobFromImage(img, 1/255, (320, 320), swapRB=1)
            net.setInput(blob)
            outs = net.forward(layerOutputs)
        except Exception as e:
            print(str(e))

        # Non-maximum Suppression
        imgH, imgW, _ = img.shape
        classIDs = []
        confidences = []
        boxes = []
        for out in outs:
            for detection in out:
                scores = list(detection[5:])
                confidence = max(scores)
                classID = scores.index(confidence)
                if confidence > TH1:
                    centerX, centerY   = int(detection[0] * imgW), int(detection[1] * imgH)
                    boxW, boxH         = int(detection[2] * imgW), int(detection[3] * imgH)
                    topleftX, topleftY = int(centerX - boxW / 2), int(centerY - boxH / 2)
                    boxes.append([topleftX, topleftY, boxW, boxH])
                    confidences.append(float(confidence))
                    classIDs.append(int(classID))
        resultDet = cv2.dnn.NMSBoxes(boxes, confidences, TH1, TH2)

        # Visualization
        for ind in resultDet:
            id = ind[0]
            topleftX, topleftY, boxW, boxH = boxes[id]
            label = f'{classNames[classIDs[id]]}: {confidences[id]:.2}'
            if "cell phone" in label:
                cv2.putText(img, "Warning!", (100, 300), cv2.FONT_HERSHEY_PLAIN, 12.0, (0, 0, 255), 5); # 입력 이미지, 입력 문구, 문구 시작 위치, 폰트, 글자크기, 글자색상, 글자굵기
            color = classColors[classNames[classIDs[id]]]
            cv2.rectangle(img, (topleftX, topleftY, boxW, boxH), color, 2)
            cv2.putText(img, label, (topleftX, topleftY-12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
        eTime, _ = net.getPerfProfile()
        # print("Elapsed time: %.2f ms" % (eTime * 1000.0 / cv.getTickFrequency()))
        
        cv2.imwrite(f"/content/drive/MyDrive/Colab Notebooks/ComputerVision/project/result/{now_frame:.0f}.jpg", img)

100%|██████████| 320/320 [09:55<00:00,  1.86s/it]


In [ ]:
import glob

img_array = []

for file in tqdm(glob.glob('/content/drive/MyDrive/Colab Notebooks/ComputerVision/project/result/*.jpg')):
    img = cv2.imread(file)
    img_array.append(img)

height, width = img.shape[:2]
size = (width, height)
video = cv2.VideoWriter('/content/drive/MyDrive/Colab Notebooks/ComputerVision/project/result/result.avi',cv2.VideoWriter_fourcc(*'DIVX'), 20, size)

for img in img_array:
    video.write(img)

video.release()

100%|██████████| 320/320 [00:03<00:00, 97.10it/s]


# APPENDIX
### 외부 데이터 출처
    
* https://github.com/seongheum-ssu/2021-1-2150054901/blob/main/exp/2021-05-13